# Методы сбора и обработки данных из сети Интернет
## Урок 3. Парсинг данных. HTML, Beautiful Soap
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

* Наименование вакансии.
* Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.
***
1. установим BeautifulSoup
2. импротируем библиотеки

In [55]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [56]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
from time import sleep

парсинг будет выполнятся сайта https://hh.ru
для того чтоб сайт не заблокировал пропишем актуальный User-Agent

спарсим информацию с сайта и переведем ее в text формат

In [57]:
name = input('Введите название професии')
url = 'https://hh.ru/search/vacancy?text=' + name
print(url)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.2.928 Yowser/2.5 Safari/537.36',
}
response = requests.get(url=url, headers=headers)
response.text
#supernova-search-group__input

Введите название професииData Analyst
https://hh.ru/search/vacancy?text=Data Analyst


'<!DOCTYPE html>\n<html class="desktop" lang="ru"><!--request: 1676878603072f508ccdfbb6148a8dba--><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><title data-rh="" data-reactroot="">Работа в Волжском (Волгоградская область), поиск персонала и публикация вакансий - volzhskiy.hh.ru</title><meta name="twitter:card" content="summary_large_image"><meta data-rh="" name="description" content="volzhskiy.hh.ru — сервис, который помогает найти работу и подобрать персонал в Волжском (Волгоградская область) более 20 лет! Создавайте резюме и откликайтесь на вакансии. Набирайте сотрудников и публикуйте вакансии." data-reactroot=""><meta data-rh="" name="keywords" content="работа, вакансии, работа, поиск вакансий, резюме, работы, работу, работ, ищу работу, поиск" data-reactroot=""><meta property="og:image" content="https://i.hh.ru/logos/openGraph/hh.ru.png?v=12012023?v=3"><meta property="og:image:width" content="600"><meta 

In [58]:
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }


In [64]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result

In [60]:
def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [65]:
result = parse_hh(url, headers)

Cтраница №1, ссылка: https://volzhskiy.hh.ru/search/vacancy?text=Data%20Analyst
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=4&hhtmFrom=vacancy_search_list
Cтраница №6, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=5&hhtmFrom=vacancy_search_list
Cтраница №7, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=6&hhtmFrom=vacancy_search_list
Cтраница №8, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=7&hhtmFrom=vacancy_search_list
Cтраница №9, ссылка: https://spb.hh.ru/search/vacancy?text=Data+Analyst&page=8&hhtmFrom=vacancy_search_list
Cтраница №10, ссылка: https://spb.hh.ru/search/vacancy?t

In [66]:
len(result)

800

In [67]:
result

[{'vacancy_name': 'Аналитик данных / Data Analyst',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://volzhskiy.hh.ru/vacancy/69166398?from=vacancy_search_list&query=Data+Analyst',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Аналитик данных / Data Analyst',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://volzhskiy.hh.ru/vacancy/76869170?from=vacancy_search_list&query=Data+Analyst',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Продуктовый аналитик-исследователь',
  'vacancy_salary': 'от 150\u202f000 руб.',
  'min_salary': 150000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://volzhskiy.hh.ru/vacancy/73265382?from=vacancy_search_list&query=Data+Analyst',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Junior Data Analyst',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'curre

In [72]:
import pandas as pd

In [73]:
chek = pd.DataFrame(result)
chek

,vacancy_name,vacancy_salary,min_salary,max_salary,currency_salary,vacancy_link,vacancy_source
0,Аналитик данных / Data Analyst,None,NaN,NaN,None,https://volzhskiy.hh.ru/vacancy/69166398?from=...,hh.ru
1,Аналитик данных / Data Analyst,None,NaN,NaN,None,https://volzhskiy.hh.ru/vacancy/76869170?from=...,hh.ru
2,Продуктовый аналитик-исследователь,от 150 000 руб.,150000.0,NaN,руб.,https://volzhskiy.hh.ru/vacancy/73265382?from=...,hh.ru
3,Junior Data Analyst,None,NaN,NaN,None,https://volzhskiy.hh.ru/vacancy/77191771?from=...,hh.ru
4,Data Analyst,2 000 – 5 000 USD,2000.0,5000.0,USD,https://volzhskiy.hh.ru/vacancy/76354970?from=...,hh.ru
...,...,...,...,...,...,...,...
795,Консультант по бюджетированию и казначейству 1...,None,NaN,NaN,None,https://spb.hh.ru/vacancy/76322662?from=vacanc...,hh.ru
796,Трафик менеджер / маркетолог (удаленно),от 80 000 руб.,80000.0,NaN,руб.,https://spb.hh.ru/vacancy/76579082?from=vacanc...,hh.ru
797,.NET/C# Разработчик (Remote),до 350 000 руб.,NaN,350000.0,руб.,https://spb.hh.ru/vacancy/76585431?from=vacanc...,hh.ru
798,SEO-специалист / Специалист по поисковой оптим...,None,NaN,NaN,None,https://spb.hh.ru/vacancy/76610828?from=vacanc...,hh.ru
